In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv("UNSW_NB15_training-set.csv")
test_df = pd.read_csv("UNSW_NB15_testing-set.csv")

In [ ]:
target_col = 'label'
categorical_cols = ['proto', 'service', 'state']
numeric_cols = [col for col in train_df.columns if col not in categorical_cols + [target_col, 'attack_cat', 'id']]

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = encoder.fit_transform(train_df[categorical_cols])
X_test_cat = encoder.transform(test_df[categorical_cols])


scaler = StandardScaler()
X_train_num = scaler.fit_transform(train_df[numeric_cols])
X_test_num = scaler.transform(test_df[numeric_cols])

In [ ]:
X_train_combined = np.hstack((X_train_num, X_train_cat))
X_test_combined = np.hstack((X_test_num, X_test_cat))


X_train_seq = X_train_combined.reshape((X_train_combined.shape[0], X_train_combined.shape[1], 1))
X_test_seq = X_test_combined.reshape((X_test_combined.shape[0], X_test_combined.shape[1], 1))

In [ ]:
y_train = train_df[target_col].values
y_test = test_df[target_col].values

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_seq.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=64))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_seq, y_train, epochs=10, batch_size=64, validation_data=(X_test_seq, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.8515 - loss: 0.3157 - val_accuracy: 0.8294 - val_loss: 0.2804
Epoch 2/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 37s 10ms/step - accuracy: 0.9315 - loss: 0.1562 - val_accuracy: 0.8148 - val_loss: 0.3028
Epoch 3/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 39s 9ms/step - accuracy: 0.9278 - loss: 0.1632 - val_accuracy: 0.8165 - val_loss: 0.2844
Epoch 4/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - accuracy: 0.9327 - loss: 0.1430 - val_accuracy: 0.8109 - val_loss: 0.3040
Epoch 5/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 45s 11ms/step - accuracy: 0.9335 - loss: 0.1345 - val_accuracy: 0.8523 - val_loss: 0.2584
Epoch 6/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 36s 9ms/step - accuracy: 0.9357 - loss: 0.1295 - val_accuracy: 0.8337 - val_loss: 0.3096
Epoch 7/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.9386 - loss: 0.1274 - val_accuracy: 0.8863 - val_loss: 0.2531
Epoch 8/10
2740/2740 ━━━━━━━━━━━━━━━━━━━━ 43s 10ms/step - accuracy: 0.9413 - lo

In [ ]:
y_pred_probs = model.predict(X_test_seq).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_pred_probs)

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
fpr = fp / (fp + tn + 1e-6)
fnr = fn / (fn + tp + 1e-6)

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step


In [ ]:
print(f"Accuracy     : {acc:.4f}")
print(f"F1-Score     : {f1:.4f}")
print(f"ROC-AUC      : {roc:.4f}")
print(f"False Pos Rate (FPR): {fpr:.4f}")
print(f"False Neg Rate (FNR): {fnr:.4f}")

Accuracy     : 0.8711
F1-Score     : 0.8939
ROC-AUC      : 0.9771
False Pos Rate (FPR): 0.2701
False Neg Rate (FNR): 0.0136
